# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [8]:
!pip install --upgrade gensim

In [1]:
# TODO: import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary

Load the data in the file `random_headlines.csv`

In [2]:
# TODO: load the dataset
df = pd.read_csv("random_headlines.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [3]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [4]:
# TODO: Preprocess the input data
df["tokenize"] = df["headline_text"].apply(lambda row: word_tokenize(row))
df["alpha"] = df["tokenize"].apply(lambda row:[word for word in row if word.isalpha()])

stop_words = set(stopwords.words('english'))
df["stop"] = df["alpha"].apply(lambda row:[word for word in row if word not in stop_words])

stemmer = PorterStemmer()

df["stem"] = df["stop"].apply(lambda row:[stemmer.stem(word) for word in row ])
df["stem"].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stem, dtype: object

Now use Gensim to compute a BOW

In [5]:
# TODO: Compute the BOW using Gensim
dct = Dictionary(df["stem"])
df['BOW'] = df["stem"].apply(lambda document: dct.doc2bow(document)) 
df['BOW']

0                 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
1                                 [(5, 1), (6, 1), (7, 1)]
2                       [(8, 1), (9, 1), (10, 1), (11, 1)]
3                     [(12, 1), (13, 1), (14, 1), (15, 1)]
4                     [(16, 1), (17, 1), (18, 1), (19, 1)]
                               ...                        
19995    [(121, 1), (743, 1), (983, 1), (9722, 1), (122...
19996    [(378, 1), (610, 1), (1442, 1), (1663, 1), (22...
19997                       [(154, 1), (446, 1), (535, 1)]
19998         [(163, 1), (3943, 1), (6310, 1), (12209, 1)]
19999    [(948, 1), (1741, 1), (1993, 1), (2717, 1), (3...
Name: BOW, Length: 20000, dtype: object

In [6]:
corpus_bow = [dct.doc2bow(document) for document in df["stem"]]
print(np.shape(corpus_bow))
corpus_bow[:2]

(20000,)


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

Compute the TF-IDF using Gensim

In [7]:
# TODO: Compute TF-IDF
tfidf_model = TfidfModel(df['BOW'].to_list())
tfidf = tfidf_model[df['BOW'].to_list()]

print(np.shape(tfidf))
tfidf

(20000,)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [40]:
tfidf[0]

[(0, 0.30725466582280214),
 (1, 0.3528943781678455),
 (2, 0.42129048115131124),
 (3, 0.5992666854471201),
 (4, 0.49442279315598586)]

In [8]:
# TODO: Compute LSA
lsi_model = LsiModel(df['BOW'].to_list(),id2word=dct, num_topics = 4)
lsi_model

For each of the topic, show the most significant words.

In [9]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi_model.print_topics(num_words=3)

[(0, '0.752*"polic" + 0.405*"man" + 0.208*"charg"'),
 (1, '-0.670*"man" + 0.575*"polic" + -0.329*"charg"'),
 (2, '0.656*"new" + 0.296*"plan" + 0.241*"say"'),
 (3, '0.702*"new" + -0.339*"say" + -0.333*"plan"')]

What do you think about those results?

There is only a slightly difference between the topics, with the significant words being more or less the same.

Now let's try to use LDA instead of LSA using Gensim

In [10]:
# TODO: Compute LDA
lda_model = LdaModel(df['BOW'].to_list(),id2word=dct, num_topics = 4)
lda_model

In [11]:
# TODO: print the most frequent words of each topic
lda_model.print_topics(num_words=3)

[(0, '0.010*"kill" + 0.008*"interview" + 0.007*"win"'),
 (1, '0.013*"charg" + 0.013*"man" + 0.013*"polic"'),
 (2, '0.009*"polic" + 0.009*"new" + 0.007*"council"'),
 (3, '0.005*"fire" + 0.005*"report" + 0.005*"water"')]

Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [49]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 6.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=474cce18e90ea672085985dd1f5bb7c758c0d73e1b1f48b8ec801cbed6a1be33
  Stored in directory: /Users/laravaroni/Library/Caches/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=43261be86c046788abdb2a6db3903e9e6102e6c157a5993a370776d065dc973b
  Stored in directory: /Users/laravaroni/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built pyLDAvis sklearn
Note: you may need to restart the kernel to use updated packages.


In [14]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, df['BOW'].to_list(), dct)
vis

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.073835 -0.025662       1        1  28.293791
1      0.100068 -0.161340       2        1  24.899639
0     -0.234666 -0.022601       3        1  23.522183
3      0.060762  0.209603       4        1  23.284388, topic_info=      Term        Freq       Total Category  logprob  loglift
86   charg  339.000000  339.000000  Default  30.0000  30.0000
688   kill  264.000000  264.000000  Default  29.0000  29.0000
139    man  466.000000  466.000000  Default  28.0000  28.0000
185  polic  631.000000  631.000000  Default  27.0000  27.0000
312  court  288.000000  288.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
155    new  122.482434  482.596014   Topic4  -5.2732   0.0862
124     us   98.073429  263.699508   Topic4  -5.4954   0.4683
84   trial   73.866821  140.622580   Topic4  -5.7789   0.8136
279   work   70.980560  118.544118   Topic4  -5.8187   0.9445
305    set   71.682960  196.935800   Topic4  -5.8089   0.4468

[318 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
496       2  0.008939      abc
496       4  0.983273      abc
101       1  0.026297    accus
101       2  0.966409    accus
101       4  0.006574    accus
...     ...       ...      ...
251       1  0.237149     year
251       2  0.013175     year
251       3  0.500648     year
251       4  0.250324     year
3465      1  0.988693  zealand

[626 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.